In [6]:
import sys         

import pandas as pd
import numpy as np
from scipy.stats import iqr

from ppc_db import postgresql_extractdata

sys.path.append(r'C:\Users\jamie\OneDrive\Python\Py_24\Git_master\econ')
import fred

db_args = {
    'host':'localhost',
    'database':'LandRegData',
    'user':'root',
    'password':'password'
 }

In [7]:
cols = ['region', 'price_gbp']

query = f"SELECT {", ".join(str(x) for x in cols)} FROM Oct24pricepaidcomplete INNER JOIN county_region ON oct24pricepaidcomplete.county=county_region.county WHERE EXTRACT(YEAR FROM date)=2023"
df = pd.DataFrame(postgresql_extractdata(query, **db_args),
                  columns=['Month' if x=='EXTRACT(MONTH FROM date)' else 'Year' if x=='EXTRACT(YEAR FROM date)' else x for x in cols])


In [8]:
# PostgreSQL Quantiles

cols = ['EXTRACT(YEAR FROM date)', 'region',
        'percentile_disc(0.25) WITHIN GROUP (order by price_gbp)',
        'percentile_disc(0.50) WITHIN GROUP (order by price_gbp)',
        'percentile_disc(0.75) WITHIN GROUP (order by price_gbp)']

query = f"SELECT {", ".join(str(x) for x in cols)} from oct24pricepaidcomplete INNER JOIN county_region ON oct24pricepaidcomplete.county=county_region.county GROUP BY EXTRACT(YEAR FROM date), region"

df = pd.DataFrame(postgresql_extractdata(query, **db_args),
                      columns=['Year','region','0.25','0.50','0.75']).set_index('Year')

#SELECT EXTRACT(YEAR FROM date), region from oct24pricepaidcomplete INNER JOIN county_region ON oct24pricepaidcomplete.county=county_region.county GROUP BY EXTRACT(YEAR FROM date), region


In [9]:
df

,region,0.25,0.50,0.75
Year,,,,
1995,East Midlands,35000,45950,64000
1995,East of England,42250,57500,80250
1995,London,53500,72995,107000
1995,North East,30000,42250,59500
1995,North West,32500,45000,61335
...,...,...,...,...
2024,South East,265000,365000,515000
2024,South West,220000,297600,415000
2024,Wales,135000,191046,280000


In [10]:
# Nominal, Median

df_screened = (
    df[df['region']
       .isin(['London','South East', 'East Midlands','North West','North East'])]
       .groupby(['Year','region'])['0.50'].sum().unstack()
)

import plotly.express as px
fig = px.line(df_screened.apply(lambda x:x/x.iloc[0]),
              labels={
                  'index':'Year',
                  'value':'Relative Price'
              },
              width=800, height=600,
              title='Median House Prices, Nominal GBP')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [11]:
# Nominal, IQR

df_screened = (
    df[df['region']
       .isin(['London','South East', 'East Midlands','North West','North East'])]
       .groupby(['Year','region']).sum().unstack()
)

import plotly.express as px
fig = px.line(df_screened['0.75']-df_screened['0.25'],
              labels={
                  'index':'Year',
                  'value':'Spread, GBP'
              },
              width=800, height=600,
              title='House Price IQR, Nominal GBP')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [13]:
df_screened = (
    df[df['region']
       .isin(['London','South East', 'East Midlands','North West','North East'])]
       .groupby(['Year','region'])['0.50'].sum().unstack()
)

gbp_cpi = (
    fred.Fred_Pull('GBRCPIALLMINMEI',
                   '899901ba06f09b9961a73113b1834a15').index(1995)
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

import plotly.express as px
fig = px.line(df_screened.mul(gbp_cpi.values,axis=1)
              .apply(lambda x: x/x.iloc[0],axis=0),
              labels={
                  'index':'Year',
                  'value':'Relative Price'
              },
              width=800, height=600,
              title='Median House Prices, CPI Adjusted GBP (1995)')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [14]:
# Post-GFC

df_screened = (
    df[df['region']
       .isin(['London','South East', 'East Midlands','North West','North East'])]
       .groupby(['Year','region'])['0.50'].sum().unstack()
       .loc[2007:]
)

gbp_cpi = (
    fred.Fred_Pull('GBRCPIALLMINMEI',
                   '899901ba06f09b9961a73113b1834a15').index(2007)
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

import plotly.express as px
fig = px.line(df_screened.mul(gbp_cpi.values,axis=1).apply(lambda x: x/x.iloc[0],axis=0),
              labels={
                  'index':'Year',
                  'value':'Relative Price'
              },
              width=800, height=600,
              title='Median House Prices, CPI Adjusted GBP (2007)')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [15]:
# Post-GFC, Real US$

import plotly.express as px

df_screened = (
    df[df['region']
       .isin(['London','South East', 'East Midlands','North West','North East'])]
       .groupby(['Year','region'])['0.50'].sum().unstack()
)

us_cpi = (
    fred.Fred_Pull('CPIAUCSL',
                   '899901ba06f09b9961a73113b1834a15').index(1995)
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

gbpusd_fx = (
    fred.Fred_Pull('DEXUSUK',
                   '899901ba06f09b9961a73113b1834a15').extract(1995)
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

usdreal = (gbpusd_fx.loc[1995:]
           .join(us_cpi)
           .assign(real=lambda df:df['DEXUSUK']*df['CPIAUCSL'])
)


fig = px.line(df_screened.mul(usdreal.real,axis=0).apply(lambda x: x/x.iloc[0]),
              labels={
                  'index':'Year',
                  'value':'Relative Price'
              },
              width=800, height=600,
              title='Median House Prices, CPI Adjusted US$ (1995)')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

Real Wages

In [20]:
# Average Weekly Earnings Per Person in the United Kingdom (FROM Q1 1919)

nominal_wages_gbp = (
    fred.Fred_Pull('AWEPPUKQ',
                   '899901ba06f09b9961a73113b1834a15').extract()
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

# Residential Property Prices for United Kingdom (FROM Q2 1846)

nominal_resproperty_gbp = (
    fred.Fred_Pull('HPIUKA',
                   '899901ba06f09b9961a73113b1834a15').extract()
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

# Household Variable Mortgage Rate in the United Kingdom (FROM 1853)

nominal_mortrate_perc = (
    fred.Fred_Pull('CMRAUKA',
                   '899901ba06f09b9961a73113b1834a15').extract()
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

# Consumer Price Indices ..., COICOP 1999: Consumer Price Index... (FROM 1914)

gbp_cpi = (
    fred.Fred_Pull('CPIUKQ',
                   '899901ba06f09b9961a73113b1834a15').extract()
                   .apply(lambda x:1/(x/x.iloc[-1]))
                   .resample('YE').mean()
                   .assign(year=lambda df: df.index.year).set_index('year')
)

In [21]:
df = (
    nominal_wages_gbp.join([nominal_resproperty_gbp,nominal_mortrate_perc,gbp_cpi])
    .assign(real_prop_gbp = lambda df:df['HPIUKA']*df['CPIUKQ'],
            real_wages_gbp = lambda df: df['AWEPPUKQ']*df['CPIUKQ'],
            nominal_mortrate_perc = lambda df: df['CMRAUKA']
            )
)

#df=df.loc[df.index<2007]

In [27]:
df[:2008]

,AWEPPUKQ,HPIUKA,CMRAUKA,CPIUKQ,real_prop_gbp,real_wages_gbp,nominal_mortrate_perc
year,,,,,,,
1919,3.1200,0.15,5.25,25.855503,3.878325,80.669170,5.25
1920,3.7225,0.18,6.25,22.168684,3.990363,82.522928,6.25
1921,3.7150,0.21,6.75,25.458823,5.346353,94.579526,6.75
1922,2.8700,0.21,6.53,30.696461,6.446257,88.098844,6.53
1923,2.5650,0.23,6.30,32.080812,7.378587,82.287284,6.30
...,...,...,...,...,...,...,...
2012,468.5000,88.40,3.38,1.056861,93.426555,495.139605,3.38
2013,474.0000,90.68,3.33,1.030385,93.435291,488.402382,3.33
2014,479.8325,97.96,3.22,1.015518,99.480096,487.278308,3.22


In [26]:
import seaborn as sns

fig = px.scatter(df[:2008],x='real_wages_gbp',y='real_prop_gbp',
              labels={
                  'real_wages_gbp':'Average Weekly Earnings',
                  'real_prop_gbp':'Residential Property Price Index'
              },
              width=800, height=600, trendline="ols",
              title='UK House Price Index vs. Weekly Earnings, CPI Adjusted GBP (2024)')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [28]:
fig = px.scatter(df[:2008],x='nominal_mortrate_perc',y='real_prop_gbp',
              labels={
                  'nominal_mortrate_perc':'Nominal Mortgage Rate, %',
                  'real_prop_gbp':'Residential Property Price Index, CPI Adjusted GBP'
              },
              width=800, height=600, trendline="ols",
              title='UK House Price Index vs. Nominal Mortgage Rates')

fig.update_layout(legend=dict(
    title='Region',
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [29]:
import pandas as pd # 0.23.0
import requests     # 2.19.1
import io

url_endpoint = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'

payload = {
    'Datefrom'   : '01/Jan/1995',
    'Dateto'     : '02/Nov/2024',
    'SeriesCodes': 'RPQZ4TL',
    'CSVF'       : 'TN',
    'UsingCodes' : 'Y',
    'VPD'        : 'Y',
    'VFD'        : 'N'
}



headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/54.0.2840.90 '
                  'Safari/537.36'
}

response = requests.get(url_endpoint, params=payload, headers=headers)


# Check if the response was successful, it should return '200'
print(response.status_code)

df = pd.read_csv(io.BytesIO(response.content)).set_index('DATE')


200
